In [1]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib widget



In [2]:
# Read csv and save to parquet. Needs to be done only once
# transactions = pd.read_csv('data/synt_transactions_10M.csv', sep=';')
# transactions.to_parquet('data/transactions_categorical.parquet')

In [5]:
transactions = pd.read_parquet('data/transactions_categorical.parquet')

In [3]:
example = transactions.loc[transactions['tilinro'] == 1]


In [4]:
example.loc[:, 'maksupvm'] = pd.to_datetime(example['maksupvm'])

/home/juho/.pyenv/versions/3.8.6/envs/junction_2020/lib/python3.8/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [5]:
plt.plot(example['maksupvm'], example['saldo'])
plt.show()
plt.

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …